In [1]:
try:
    import os, uuid, sys, boto3, time, sys
    from pyspark.sql.functions import lit, udf
    import sys
    from pyspark.context import SparkContext
    from pyspark.sql.session import SparkSession
    from awsglue.transforms import *
    from awsglue.utils import getResolvedOptions
    from pyspark.context import SparkContext
    from pyspark.sql.functions import col, explode, lit, concat
    from awsglue.context import GlueContext
    from awsglue.dynamicframe import DynamicFrame
    from awsglue.job import Job
except Exception as e:
    print("Modules are missing : {} ".format(e))

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Create a Spark context and Glue context
sc = spark.sparkContext
glueContext = GlueContext(sc)
job = Job(glueContext)
logger = glueContext.get_logger()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/home/glue_user/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(

In [3]:
## fixtures

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
glue_df = glueContext.create_dynamic_frame.from_catalog(
            database="football_db",
            table_name="fixtures"
        )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [124]:
fixtures_df = glue_df.toDF()
fixtures_response_df = fixtures_df.select("response")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [125]:
# Explode the array to create a new row for each JSON object in the array
fixtures_exploded_df = fixtures_response_df.select(explode("response").alias("fixture"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [126]:
fixtures_final_df = fixtures_exploded_df.select(
    col("fixture.fixture.id").alias("fixture_id"),
    col("fixture.fixture.referee").alias("referee"),
    col("fixture.fixture.timezone").alias("timezone"),
    col("fixture.fixture.date").alias("date"),
    col("fixture.fixture.venue.name").alias("Stadium"),
    col("fixture.teams.home.name").alias("Home team"),
    col("fixture.teams.away.name").alias("Away team"),
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [127]:
fixtures_dynamic_frame = DynamicFrame.fromDF(fixtures_final_df, glueContext, "fixtures_dynamic_frame")

# Specify the S3 output path where you want to store the CSV file
output_path = "s3://footballapi-data-bucket/processed/fixtures"

# Write the DynamicFrame to S3 as a CSV file
glueContext.write_dynamic_frame.from_options(
    frame = fixtures_dynamic_frame,
    connection_type = "s3",
    connection_options = {"path": output_path},
    format = "csv",
    format_options = {"writeHeader": True},
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [128]:
##results

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [129]:
glue_df = glueContext.create_dynamic_frame.from_catalog(
            database="football_db",
            table_name="results"
        )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
results_df = glue_df.toDF()
results_response_df = results_df.select("response")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
results_exploded_df = results_response_df.select(explode("response").alias("results"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
results_final_df = results_exploded_df.select(
    col("results.fixture.id").alias("fixture_id"),
    col("results.fixture.referee").alias("referee"),
    col("results.fixture.timezone").alias("timezone"),
    col("results.fixture.date").alias("date"),
    col("results.fixture.venue.name").alias("Stadium"),
    col("results.teams.home.name").alias("Home team"),
    col("results.teams.away.name").alias("Away team"),
    col("results.goals.home").alias("Home goals"),
    col("results.goals.away").alias("Away goals"),
    col("results.score.halftime.Home").alias("Half time Home goals"),
    col("results.score.halftime.away").alias("Half time Away goals"),
)
columns_to_drop = ["Home goals","Away goals", "Half time Home goals", "Half time Away goals"]
results_final_df = results_final_df.withColumn("Full time score", concat(col("Home goals"), lit(":"), col("Away Goals")))
results_final_df = results_final_df.withColumn("Half time score", concat(col("Half time Home goals"), lit(":"), col("Half time Away goals")))
results_final_df = results_final_df.drop(*columns_to_drop)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
results_dynamic_frame = DynamicFrame.fromDF(results_final_df, glueContext, "results_dynamic_frame")

# Specify the S3 output path where you want to store the CSV file
output_path = "s3://footballapi-data-bucket/processed/results"

# Write the DynamicFrame to S3 as a CSV file
glueContext.write_dynamic_frame.from_options(
    frame = results_dynamic_frame,
    connection_type = "s3",
    connection_options = {"path": output_path},
    format = "csv",
    format_options = {"writeHeader": True},
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [134]:
##standings

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [135]:
glue_df = glueContext.create_dynamic_frame.from_catalog(
            database="football_db",
            table_name="standings"
        )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [136]:
standings_df = glue_df.toDF()
standings_response_df = standings_df.select("response")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [137]:
# Explode the array to create a new row for each JSON object in the array
standings_exploded_df = standings_response_df.select(explode("response.league.standings").alias("standings"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [138]:
standings_exploded_df.show(truncate=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|           standings|
+--------------------+
|[[{1, {50, Manche...|
+--------------------+

In [139]:
standings_exploded_df = standings_response_df.select(
    explode("response.league.standings").alias("league"),
).select(
    explode("league").alias("standings")
).select(
    explode("standings").alias("standings")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|           standings|
+--------------------+
|{1, {50, Manchest...|
|{2, {42, Arsenal,...|
|{3, {47, Tottenha...|
|{4, {40, Liverpoo...|
|{5, {66, Aston Vi...|
|{6, {33, Manchest...|
|{7, {34, Newcastl...|
|{8, {51, Brighton...|
|{9, {55, Brentfor...|
|{10, {49, Chelsea...|
|{11, {39, Wolves,...|
|{12, {52, Crystal...|
|{13, {48, West Ha...|
|{14, {45, Everton...|
|{15, {65, Notting...|
|{16, {36, Fulham,...|
|{17, {35, Bournem...|
|{18, {1359, Luton...|
|{19, {44, Burnley...|
|{20, {62, Sheffie...|
+--------------------+

In [140]:
standings_final_df = standings_exploded_df.select(
    col("standings.rank").alias("League Position"),
    col("standings.team.name").alias("Team"),
    col("standings.points").alias("League Points"),
    col("standings.all.win").alias("Wins"),
    col("standings.all.draw").alias("Draws"),
    col("standings.all.lose").alias("Loses"),
    col("standings.goalsDiff").alias("Goal Difference"),
    col("standings.all.goals.for").alias("Goals For"),
    col("standings.all.goals.against").alias("Goals Against"),
    col("standings.form").alias("Form"),
    col("standings.away.win").alias("Away wins"),
    col("standings.home.lose").alias("Home loses"),    
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [141]:
standings_dynamic_frame = DynamicFrame.fromDF(standings_final_df, glueContext, "standings_dynamic_frame")

# Specify the S3 output path where you want to store the CSV file
output_path = "s3://footballapi-data-bucket/processed/standings"

# Write the DynamicFrame to S3 as a CSV file
glueContext.write_dynamic_frame.from_options(
    frame = standings_dynamic_frame,
    connection_type = "s3",
    connection_options = {"path": output_path},
    format = "csv",
    format_options = {"writeHeader": True},
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [142]:
job.commit()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
##goal scorers

In [ ]:
glue_df = glueContext.create_dynamic_frame.from_catalog(
            database="football_db",
            table_name="scorers"
        )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
scorers_df = glue_df.toDF()
scorers_response_df = scorers_df.select("response")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
# Explode the array to create a new row for each JSON object in the array
scorers_exploded_df = scorers_response_df.select(explode("response").alias("topscorers"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
scorers_final_df = scorers_exploded_df.select(
    col("topscorers.player.name").alias("Name"),
    col("topscorers.statistics.goals.total").alias("Goals scored"),
    col("topscorers.statistics.shots.on").alias("Shots on Target"),
    col("topscorers.statistics.shots.total").alias("Shots"),
    col("topscorers.statistics.penalty.scored").alias("Penalties scored"),
)

scorers_final_df = scorers_final_df.withColumn("Shots on Target", col("Shots on Target").getItem(0))
scorers_final_df = scorers_final_df.withColumn("Goals scored", col("Goals scored").getItem(0))
scorers_final_df = scorers_final_df.withColumn("Shots", col("Shots").getItem(0))
scorers_final_df = scorers_final_df.withColumn("Penalties scored", col("Penalties scored").getItem(0))



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
scorers_dynamic_frame = DynamicFrame.fromDF(scorers_final_df, glueContext, "scorers_dynamic_frame")

# Specify the S3 output path where you want to store the CSV file
output_path = "s3://footballapi-data-bucket/processed/scorers"

# Write the DynamicFrame to S3 as a CSV file
glueContext.write_dynamic_frame.from_options(
    frame = scorers_dynamic_frame,
    connection_type = "s3",
    connection_options = {"path": output_path},
    format = "csv",
    format_options = {"writeHeader": True},
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
##assists

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
glue_df = glueContext.create_dynamic_frame.from_catalog(
            database="football_db",
            table_name="assists"
        )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
assists_df = glue_df.toDF()
assists_response_df = assists_df.select("response")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
# Explode the array to create a new row for each JSON object in the array
assists_exploded_df = assists_response_df.select(explode("response").alias("assists"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
assists_final_df = assists_exploded_df.select(
    col("assists.player.name").alias("Name"),
    col("assists.statistics.goals.assists").alias("Assists"),
    col("assists.statistics.passes.key").alias("Key Passes"),
    col("assists.statistics.shots.on").alias("Shots on Target"),
    col("assists.statistics.passes.total").alias("Total successfull passes"),
)
assists_final_df = assists_final_df.withColumn("Assists", col("Assists").getItem(0))
assists_final_df = assists_final_df.withColumn("Key Passes", col("Key Passes").getItem(0))
assists_final_df = assists_final_df.withColumn("Shots on Target", col("Shots on Target").getItem(0))
assists_final_df = assists_final_df.withColumn("Total successfull passes", col("Total successfull passes").getItem(0))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
assists_dynamic_frame = DynamicFrame.fromDF(assists_final_df, glueContext, "assists_dynamic_frame")

# Specify the S3 output path where you want to store the CSV file
output_path = "s3://footballapi-data-bucket/processed/assists"

# Write the DynamicFrame to S3 as a CSV file
glueContext.write_dynamic_frame.from_options(
    frame = assists_dynamic_frame,
    connection_type = "s3",
    connection_options = {"path": output_path},
    format = "csv",
    format_options = {"writeHeader": True},
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
job.commit()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…